# WEB SCRAPING PRACTICE - WEB SCRAPER

## Libraries

In [4]:
import pandas as pd
from bs4 import BeautifulSoup # Librería para crear objetos tipo soup
import urllib3 # Librería para descargar el contenido de una URL definida
import numpy as np # Librería para manipular arreglos
from sqlalchemy import create_engine # Librería para crear las conexiones con la base de datos Postgre
from datetime import datetime
from datetime import timedelta, time
import pickle
import requests
import sys
import re
import random # Librería para generar números randómicos

## Supporting Proxies

In [6]:
# Argumento que proviene del proceso de extracción de proxies
proxies = {
  'http': 'http://{}:{}'.format(proxy_list[proxy_index],port_list[proxy_index]),
  'https': 'http://{}:{}'.format(proxy_list[proxy_index],port_list[proxy_index])}

In [7]:
proxies

## Functions

In [8]:
# Función download() para realizar la petición GET a una URL especificada
def download():
    # Aplicamos el módulo urllib3 para descargar una URL definida
    num_retries = 5 # Número máximo de reintentos a efecuar en caso de error en la descarga
    try:
        http = urllib3.PoolManager()
        response = http.request('GET', URL)
        response.status
        return response.data
    
    # En el caso de que exista un error colocamos una excepción para conocer el motivo
    except urllib.URLError  as e: 
        print ("Download error", e.reason)
        html = None
        if num_retries > 0:  
            ## Realizamos hasta 5 reintentos de la petición en caso de presentarse los errores más frecuentes
            if hasattr(e, 'code') and 400 <= e.code < 600:  
                return download(URL, num_retries-1) 
        return html

# AMAZON WEB SCRAPING PROCESS

In [9]:
''' Se realiza la captura de los precios, descripciones y códigos únicos de los productos de Amazon (mascarillas):

Para dicho objetivo se utilizan URLs de Amazon de diferentes países.

El dataframe resultante con los datos en bruto es: "dailyData"

'''

' Se realiza la captura de los precios, descripciones y códigos únicos de los productos de Amazon (mascarillas):\n\nPara dicho objetivo se utilizan URLs de Amazon de diferentes países.\n\nEl resultado será un Dataset conteniendo los datos capturados de los siguientes campos:\n\nNo hay N/A detectados\n\n\'asin\',              tipo CHR          "Amazon Standar Identification Number - Código único de identificación del producto"\n\'description\',       tipo CHR          "Descripción del producto"\n\'dateTime\'           tipo timestamp    "Fecha y hora en la que se realiza la captura de datos"\n\'date\'               tipo datetime     "Fecha en la que se realiza la captura de datos"\n\'country\'            tipo CHR          "País al que pertenece el producto "\n\'currency\'           tipo CHR          "Moneda local en la que se expresa el precio del producto"\n\'price\'              tipo FLOAT        "Precio del producto"\n'

## Amazon URL List

In [10]:
# Detalle de URL de Amazon de cada país para la extracción diaria de datos
urlSpain = 'https://www.amazon.es/s?k=MASCARILLAS+FFP2&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss'
urlUSA = 'https://www.amazon.com/s?k=FFP2+mask&ref=nb_sb_noss_2'
urlFrance = 'https://www.amazon.fr/s?k=masque+facial+FFP2&__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'
urlUK = 'https://www.amazon.co.uk/s?k=FFP2+mask&ref=nb_sb_noss_2'
urlGermany = 'https://www.amazon.de/s?k=Gesichtsmaske+FFP2&__mk_de_DE=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'
urlItaly = 'https://www.amazon.it/s?k=maschera+FFP2&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'
urlNetherlands = 'https://www.amazon.nl/s?k=gezichtsmasker+ffp2&__mk_nl_NL=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=296W8O9PQ26GE&sprefix=gezichtsmasker+%2Caps%2C320&ref=nb_sb_ss_i_4_15'
urlAustralia = 'https://www.amazon.com.au/s?k=FFP2+mask&ref=nb_sb_noss_2'

# Lista con todas las URLs previamente definidas
urlList = [urlSpain, urlUSA, urlFrance, urlUK, urlGermany, urlItaly, urlNetherlands, urlAustralia]

## Amazon Data download

In [11]:
# Lista de los países que han sido seleccionados para efectuar scraping sobre sus páginas de Amazon
countries = ['Spain', 'USA', 'France', 'UK', 'Germany', 'Italy', 'Netherlands', 'Australia']
country = 0

# Inicialización del dataframe final para almacenar todos los datos procesados
dailyData = pd.DataFrame()

# Bucle para scrapear datos por cada una de las URLs definidas previamente 
for i in urlList:   
    URL = i
    # Descarga del HTML con codificación UTF-8
    maskInfo = download()
    maskInfo = maskInfo.decode('utf-8')
    # Conversión del HTML a objeto soup
    soup = BeautifulSoup(maskInfo)
    
    # Inicialización de contadores para filas y columnas
    row = 0
    column = 0
    
    # Adquisición de la fecha actual en formato GMT
    dateToProcess = datetime.utcnow()
    
    # Dataframe para almacenar la información de una sola URL
    amazonData = pd.DataFrame({
            'asin' : "",
            'price': "",
            'description': "",
            'dateTime': dateToProcess,
        }, index = { 'indice': 0 })

    # Bucle para seleccionar todas las etiquetas ASIN, PRECIO y DESCRIPCIÓN del producto que se ofrece en Amazon
    for div in soup.select('div[data-asin]'): # Se aplican selectores CSS al objeto soup
        asin = div['data-asin']
        column += 1
        if div.select_one('.a-price'):
            price = div.select_one('.a-price ').get_text('|',strip=True).split('|')[0]
            column += 1
        if div.select_one('.a-text-normal'):
            description = div.select_one('.a-text-normal').text   
        
        # Dataframe para almacenar cada fila 
        temporalData = pd.DataFrame({
            'asin' : asin,
            'price': price,
            'description': description,
            'dateTime': dateToProcess,
        }, index = { 'indice': 0 })
        
        # Agregamos la nueva fila al dataframe que creamos para cada URL
        amazonData = amazonData.append(temporalData)
        row += 1
        column = 0
    
    # Aplicamos una primera normalización al conjunto de datos de la URL
    amazonData = amazonData.reset_index(drop=True)
    amazonData = amazonData.drop(amazonData.index[0])
    amazonData = amazonData.reset_index(drop=True)
    amazonData["date"] = amazonData["dateTime"].apply(lambda x: x.date())
    amazonData = amazonData.drop_duplicates(subset ="price")
    amazonData["country"] = countries[country]
    country += 1
    
    # Se agrega el dataframe con los datos de una URL al dataframe consolidado
    dailyData = dailyData.append(amazonData) ## DATAFRAME CON LA INFORMACIÓN RECOPILADA DESDE LA WEB DE AMAZON

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

# CORONAVIRUS WEB SCRAPING PROCESS

In [12]:
''' Vamos a captura la tabla de datos por país de afectación de la pandemia de Corona Virus en la web: 

https://www.worldometers.info/coronavirus/

El dataframe resultante con los datos en bruto es: "datos_pandemia_df" 

'''

' Vamos a captura la tabla de datos por país de afectación de la pandemia de Corona Virus en la web: \n\nhttps://www.worldometers.info/coronavirus/\n\nEl resultado será un Dataset conteniendo los datos capturados de los siguientes campos\n\nDatos de INT en unidades.\n\nNo hay N/A detectados. Los valores no detectos son CERO.\n\n\'dateTime\'           tipo timestamp    "Fecha y hora en la que se realiza la captura de datos"\n\'date\'               tipo datetime     "Fecha en la que se realiza la captura de datos"\n\'country\',           tipo CHR          "País al que hacen referencia los datos, salvo un crucero"\n\'total_cases\'        tipo INT          "Número total de casos que se han contabilizado en el país"\n\'new_cases\'          tipo INT          "Número total de casos incrementados desde la info anterior en el país"\n\'total_deaths\'       tipo INT          "Número total de fallidos en el país "\n\'new_deaths\'         tipo INT          "Número total de fallacidos incrementados 

## URL for Coronavirus Data

In [13]:
web = "https://www.worldometers.info/coronavirus/"

## Coronavirus Data download

In [14]:
# Bloque de preparación de estructuras de datos
columnas = ['dateTime','country', 'total_cases','new_cases','total_deaths','new_deaths','total_recovered','active_cases','critical_cases','cases_per_million','deaths_per_million']

# Creamos un DF vacio para contener los datos de esta captura de datos
datos_pandemia_df = pd.DataFrame(columns=columnas)
    
# Imputamos la fecha y hora como constante a emplear en la captura de datos.
dateToProcess = datetime.utcnow() ## Fecha Actual (Formato GMT)

# Bloque de captura de datos desde WEB.

# Imputamos una cabecera.
head = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# Capturamos el contenido desde la web y creamos un objeto tipo 'soup'.
r = requests.get(web, headers=head, proxies=proxies)
content = r.content
soup = BeautifulSoup(content, "html.parser")


In [15]:
# Bloque PARSING del objeto SOUP.

# Tras análisis de la estrucutra Capturamos todos las etiquetas 'td'
td_all = soup.find_all('td')

# Determinamos el largo de la tabla de forma dinámica por si cambia en el futuro
country =(soup.findAll("a", { "class" : "mt_a" }))

paises = []
for n in country:
    aux = n.text
    paises.append(aux)
    
paises = list(set(paises))
numero_paises = len(paises)

i=0
k=12

while i <= numero_paises:
   
    j= i*12
    # Capturamos Pais
    country = td_all[j+0+k].text
    
    # Capturamos total casos          
    total_cases = td_all[j+1+k].text
                
    # Capturamos nuevos_casos
    new_cases = (td_all[j+2+k].text)

    # Capturamos muertes_totales
    total_deaths = (td_all[j+3+k].text)
    
    # Capturamosnuevas_muertes
    new_deaths = (td_all[j+4+k].text)
 
    # Capturamos total recuperados
    total_recovered = (td_all[j+5+k].text)
 
    # Capturamos casos_activos
    active_cases = (td_all[j+6+k].text)
 
    # Capturamos criticos
    critical_cases = (td_all[j+7+k].text)
 
    # Capturamos casos_por millon
    cases_per_million = (td_all[j+8+k].text)

    # Capturamos muertes_por_millon
    deaths_per_million = (td_all[j+9+k].text)
 
    #append row to the dataframe

    nueva_fila = {'dateTime':dateToProcess,'country':country, 'total_cases':total_cases,'new_cases':new_cases,
            'total_deaths':total_deaths,'new_deaths':new_deaths,'total_recovered':total_recovered,
            'active_cases':active_cases,'critical_cases':critical_cases,'cases_per_million':cases_per_million,
            'deaths_per_million':deaths_per_million}
    
    #Añadimos una fila al dataset con los datos capturados

    datos_pandemia_df = datos_pandemia_df.append(nueva_fila, ignore_index=True) ## DATAFRAME CON LA INFORMACIÓN RECOPILADA DESDE LA WEB WORLDOMETERS
    
    i= i+1
